In [17]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as T
from IPython.display import clear_output
from PIL import Image
from matplotlib import cm
from time import perf_counter
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from tqdm import tqdm
import torch.optim as optim

# import optimizer as optimizer

warnings.filterwarnings('ignore')

plt.rc('font', size=30)

In [10]:
mnist_train = MNIST(
    "../datasets/mnist",
    train=True,
    download=True,
    transform=T.ToTensor()
)

In [11]:
mnist_valid = MNIST(
    "../datasets/mnist",
    train=False,
    download=True,
    transform=T.ToTensor()
)

In [12]:
train_loader = DataLoader(mnist_train, batch_size=64, shuffle=True)
valid_loader = DataLoader(mnist_valid, batch_size=64, shuffle=False)

In [13]:
def create_mlp_model():
    model = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),

        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),

        nn.Flatten(),
        nn.Linear(4 * 4 * 64, 256),
        nn.ReLU(),
        nn.Linear(256, 10)
    )
    return model

In [14]:
def evaluate(model: nn.Module, data_loader: DataLoader, loss_fn) -> tuple[float, float]:
    model.eval()

    total_loss = 0
    total = 0
    correct = 0

    for x, y in tqdm(data_loader, desc='Evaluate'):
        output = model(x)

        loss = loss_fn(output, y)

        total_loss += loss.item()

        _, y_pred = torch.max(output, 1)
        total += y.size(0)
        correct += (y_pred == y).sum().item()

    return total_loss / len(data_loader), correct / total

In [31]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = nn.CrossEntropyLoss()(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [21]:
def plot_stats(
    train_loss: list[float],
    valid_loss: list[float],
    valid_accuracy: list[float],
    title: str
):
    plt.figure(figsize=(16, 8))

    plt.title(title + ' loss')

    plt.plot(train_loss, label='Train loss')
    plt.plot(valid_loss, label='Valid loss')
    plt.legend()
    plt.grid()

    plt.show()

    plt.figure(figsize=(16, 8))

    plt.title(title + ' accuracy')

    plt.plot(valid_accuracy)
    plt.grid()

    plt.show()

In [33]:
# loading of the data
train_loader = DataLoader(mnist_train, batch_size=64, shuffle=True)
valid_loader = DataLoader(mnist_valid, batch_size=64, shuffle=False)

# Create the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = create_mlp_model().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

loss_fn = nn.CrossEntropyLoss()

In [34]:
# Train and evaluate the model
for epoch in range(1, 99):
    train(model, device, train_loader, optimizer, epoch)
    accuracy = evaluate(model, device, valid_loader)
    if accuracy >= 99:
        break

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.316147
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.292789
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.186982
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.084558
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.061846
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.091968
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.016576
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.047216
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.127303
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.033812


Evaluate: 0it [00:00, ?it/s]


TypeError: 'torch.device' object is not iterable

In [ ]:
# Save the model weights
torch.save(model.state_dict(), 'mlp_mnist.pth')

In [35]:
import torch
from torch import nn
from torch.optim import Optimizer
from torch.utils.data import DataLoader
from tqdm import tqdm

def create_mlp_model():
    model = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),

        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),

        nn.Flatten(),
        nn.Linear(4 * 4 * 64, 256),
        nn.ReLU(),
        nn.Linear(256, 10)
    )
    return model

def train(model: nn.Module, data_loader: DataLoader, optimizer: Optimizer, loss_fn):
    model.train()
    total_loss = 0
    total = 0
    correct = 0
    for x, y in tqdm(data_loader, desc='Train'):
        optimizer.zero_grad()
        output = model(x)
        loss = loss_fn(output, y)
        loss.backward()
        total_loss += loss.item()
        optimizer.step()
        _, y_pred = torch.max(output, 1)
        total += y.size(0)
        correct += (y_pred == y).sum().item()
    return total_loss / len(data_loader), correct / total

def evaluate(model: nn.Module, data_loader: DataLoader, loss_fn):
    model.eval()
    total_loss = 0
    total = 0
    correct = 0
    for x, y in tqdm(data_loader, desc='Evaluate'):
        output = model(x)
        loss = loss_fn(output, y)
        total_loss += loss.item()
        _, y_pred = torch.max(output, 1)
        total += y.size(0)
        correct += (y_pred == y).sum().item()
    return total_loss / len(data_loader), correct / total

In [37]:
import torchvision
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)



100%|██████████| 9912422/9912422 [00:00<00:00, 76840369.28it/s]

Extracting C:\Users\Alex/.pytorch/MNIST_data/MNIST\raw\train-images-idx3-ubyte.gz to C:\Users\Alex/.pytorch/MNIST_data/MNIST\raw


100%|██████████| 28881/28881 [00:00<00:00, 28883093.42it/s]

Extracting C:\Users\Alex/.pytorch/MNIST_data/MNIST\raw\train-labels-idx1-ubyte.gz to C:\Users\Alex/.pytorch/MNIST_data/MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 40172702.33it/s]

Extracting C:\Users\Alex/.pytorch/MNIST_data/MNIST\raw\t10k-images-idx3-ubyte.gz to C:\Users\Alex/.pytorch/MNIST_data/MNIST\raw



100%|██████████| 4542/4542 [00:00<00:00, 4544496.37it/s]

Extracting C:\Users\Alex/.pytorch/MNIST_data/MNIST\raw\t10k-labels-idx1-ubyte.gz to C:\Users\Alex/.pytorch/MNIST_data/MNIST\raw



In [1]:
model = create_mlp_model()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)



for epoch in range(20):
    train_loss, train_accuracy = train(model, trainloader, optimizer, loss_fn)
    print(f'Epoch {epoch+1}, Train Loss: {train_loss}, Train Accuracy: {train_accuracy}')
    test_loss, test_accuracy = evaluate(model, testloader, loss_fn)
    print(f'Epoch {epoch+1}, Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')
    if test_accuracy > 0.98:
        break

torch.save(model.state_dict(), 'mnist_model.pth')

NameError: name 'create_mlp_model' is not defined